In [ ]:
""" Summarization Training (One Shot Prompting) with Flan-T5-Small on Enriched MTS Dialogue Dataset
    https://huggingface.co/sarahahatee/t5-summarization-one-shot-better-prompt-enriched"""

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install --upgrade pip
!pip install datasets
!pip install evaluate
!pip install transformers
!pip install numpy
!pip install tensorflow
!pip install rouge_score
!pip install -U accelerate
!pip install transformers[torch]
!pip install bert_score
!pip install git+https://github.com/google-research/bleurt.git
!pip install bleurt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=d66ec16bdbb87eaff98f28e2e8d972c27e9f5004be19e2c3793e719feba45084
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import accelerate
from datasets import load_dataset, load_metric, concatenate_datasets, DatasetDict
from transformers import (
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    pipeline
)
import numpy as np
import torch._dynamo
import nltk
from huggingface_hub import notebook_login

notebook_login()


In [ ]:
# Load training datasets
train_file_a = "/content/MTS-Dialog-TrainingSet-enriched-vectorized.csv"
# train_file_b = "data/MTS-Dialog-ValidationSet.csv"

# Load testing dataset
# test_file = "data/MTS-Dialog-TestSet-1-MEDIQA-Chat-2023.csv"
test_file = "/content/MTS-Dialog-TestSet-2-MEDIQA-Sum-2023.csv"

# Load training datasets from CSV files
train_dataset_a = load_dataset('csv', data_files=train_file_a)['train']
# train_dataset_b = load_dataset('csv', data_files=train_file_b)['train']

# Load testing dataset from CSV file
test_dataset = load_dataset('csv', data_files=test_file)['train']

# Combine the two training datasets
# combined_train_dataset = concatenate_datasets([train_dataset_a, train_dataset_b])
combined_train_dataset = train_dataset_a

# Create dialog_dataset dictionary
dialog_dataset = DatasetDict({
    "train": combined_train_dataset,
    "test": test_dataset
})

metric = load_metric("rouge")

# loading flan-t5 instead of plain t5
checkpoint = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
model = checkpoint

dialog_dataset['train'][0]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

<ipython-input-4-f3e6c13a07be>:26: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

{'ID': 0,
 'section_header': 'GENHX',
 'section_text': 'The patient is a 76-year-old white female who presents to the clinic today originally for hypertension and a med check.  She has a history of hypertension, osteoarthritis, osteoporosis, hypothyroidism, allergic rhinitis and kidney stones.  Since her last visit she has been followed by Dr. Kumar.  Those issues are stable.  She has had no fever or chills, cough, congestion, nausea, vomiting, chest pain, chest pressure.',
 'dialogue': 'Doctor: What brings you back into the clinic today, miss? \r\nPatient: I came in for a refill of my blood pressure medicine. \r\nDoctor: It looks like Doctor Kumar followed up with you last time regarding your hypertension, osteoarthritis, osteoporosis, hypothyroidism, allergic rhinitis and kidney stones.  Have you noticed any changes or do you have any concerns regarding these issues?  \r\nPatient: No. \r\nDoctor: Have you had any fever or chills, cough, congestion, nausea, vomiting, chest pain, chest

In [ ]:
print(dialog_dataset['train'][0])
subset_for_one_shot = dict()
section_headers = set()
for example in dialog_dataset['train']:
    if example['section_header'] not in section_headers:
      subset_for_one_shot[example['section_header']] = ((example['dialogue'], example['section_text']))
    section_headers.add(example['section_header'])

subset_for_one_shot = DatasetDict(subset_for_one_shot)

print(subset_for_one_shot['GENHX'][0])

{'ID': 0, 'section_header': 'GENHX', 'section_text': 'The patient is a 76-year-old white female who presents to the clinic today originally for hypertension and a med check.  She has a history of hypertension, osteoarthritis, osteoporosis, hypothyroidism, allergic rhinitis and kidney stones.  Since her last visit she has been followed by Dr. Kumar.  Those issues are stable.  She has had no fever or chills, cough, congestion, nausea, vomiting, chest pain, chest pressure.', 'dialogue': 'Doctor: What brings you back into the clinic today, miss? \r\nPatient: I came in for a refill of my blood pressure medicine. \r\nDoctor: It looks like Doctor Kumar followed up with you last time regarding your hypertension, osteoarthritis, osteoporosis, hypothyroidism, allergic rhinitis and kidney stones.  Have you noticed any changes or do you have any concerns regarding these issues?  \r\nPatient: No. \r\nDoctor: Have you had any fever or chills, cough, congestion, nausea, vomiting, chest pain, chest pr

In [ ]:
MAX_INPUT_LENGTH = 1024
MAX_TARGET_LENGTH = 128


def preprocess_function(examples, include_umls_cui_mappings=False):
    """Preprocess inputs for summarization with optional inclusion of key UMLS CUI mappings."""

    if include_umls_cui_mappings and 'umls_cui_mappings' in examples:
        inputs = [f"Extract doctor notes from this patient-doctor dialogue, summarizing all the important medical information. Dialogue: {dialogue} [CUI Mappings: {cui_mappings}]\nSummary: "
                  for dialogue, cui_mappings in zip(examples["dialogue"], examples["umls_cui_mappings"])]
    else:
        inputs = [f"Extract doctor notes from this patient-doctor dialogue, summarizing all the important medical information. Dialogue: {dialogue}\nSummary: "
                  for dialogue in examples["dialogue"]]

    model_inputs = tokenizer(inputs, max_length=MAX_INPUT_LENGTH, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["section_text"], max_length=MAX_TARGET_LENGTH, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Example of processing with UMLS CUI mappings
tokenized_data_with_umls = dialog_dataset.map(
    lambda examples: preprocess_function(examples, include_umls_cui_mappings=True),
    batched=True
)

# Example output
tokenized_data_with_umls['train']
tokenizer.decode(tokenized_data_with_umls['train'][0]['input_ids'])


Map:   0%|          | 0/1201 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

"Extract doctor notes from this patient-doctor dialogue, summarizing all the important medical information. Dialogue: Doctor: What brings you back into the clinic today, miss? Patient: I came in for a refill of my blood pressure medicine. Doctor: It looks like Doctor Kumar followed up with you last time regarding your hypertension, osteoarthritis, osteoporosis, hypothyroidism, allergic rhinitis and kidney stones. Have you noticed any changes or do you have any concerns regarding these issues? Patient: No. Doctor: Have you had any fever or chills, cough, congestion, nausea, vomiting, chest pain, chest pressure? Patient: No. Doctor: Great. Also, for our records, how old are you and what race do you identify yourself as? Patient: I am seventy six years old and identify as a white female. [CUI Mappings: [('patient', ['C0030705']), ('hypertension', ['C0020538', 'C0020538', 'C0020538']), ('history', ['C0262926']), ('hypertension', ['C0020538', 'C0020538', 'C0020538']), ('osteoarthritis', ['C

In [ ]:
from bleurt import score as bleurt_score
from bert_score import score as bert_score


data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer, model=checkpoint, label_pad_token_id=-100, pad_to_multiple_of=8
)


bleurt = load_metric("bleurt", module_type="metric", checkpoint="BLEURT-20")


def compute_metrics(eval_pred):
    """Prepare evaluation metrics - Rouge, BERTScore, BLEURT-20"""
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge
    decoded_preds_rouge = [
        "\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds
    ]
    decoded_labels_rouge = [
        "\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels
    ]
    rouge_result = metric.compute(
        predictions=[decoded_preds_rouge],
        references=[decoded_labels_rouge],
        use_stemmer=True
    )
    rouge_result = {key: value.mid.fmeasure * 100 for key, value in rouge_result.items()}

    # BERTScore
    _, _, bert_scores = bert_score(decoded_preds, decoded_labels, lang="en", verbose=False)
    bert_f1 = bert_scores.mean().item()  # Take the mean BERTScore value

    # BLEURT-20
    scorer = bleurt_score.BleurtScorer(bleurt)
    bleurt_scores = scorer.score(references=decoded_labels, candidates=decoded_preds)
    mean_bleurt_score = np.mean(bleurt_scores)

    # Calculate mean generated length
    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions
    ]
    gen_len = np.mean(prediction_lens)

    return {
        'rouge': {k: round(v, 4) for k, v in rouge_result.items()},
        'bert_score': round(bert_f1, 4),
        'bleurt_20': round(mean_bleurt_score, 4),
        'gen_len': round(gen_len, 4)
    }


# load flan-t5 model
model = checkpoint

# prepare trainer arguments. tweak if performance unsatisfactory.
BATCH_SIZE = 7
args = Seq2SeqTrainingArguments(
    "t5-summarization-one-shot-better-prompt-enriched",
    evaluation_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=20,
    predict_with_generate=True,
    fp16=False,
    logging_steps=75,
    eval_steps=75,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_data_with_umls["train"],
    eval_dataset=tokenized_data_with_umls["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


torch._dynamo.config.suppress_errors = True
# to make it continue if you don't have any GPU available

# don't know what it does but it's VITAL.
nltk.download("punkt")

trainer.train()

trainer.evaluate()

trainer.push_to_hub()


# have to bring all tensors on the same machine
model.to("cpu")


/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for bleurt contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/bleurt/bleurt.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Epoch,Training Loss,Validation Loss,Rouge,Bert Score,Bleurt 20,Gen Len
1,1.065800,2.543947,"{'rouge1': 42.7473, 'rouge2': 18.8594, 'rougeL': 19.0931, 'rougeLsum': 19.0931}",0.875000,-0.824600,13.475000
2,1.029900,2.608090,"{'rouge1': 41.8654, 'rouge2': 18.8191, 'rougeL': 19.2938, 'rougeLsum': 19.2938}",0.874900,-0.804400,13.590000
3,0.969600,2.614318,"{'rouge1': 42.028, 'rouge2': 19.0578, 'rougeL': 19.2662, 'rougeLsum': 19.2662}",0.874200,-0.832900,13.410000
4,1.239900,2.486242,"{'rouge1': 45.2998, 'rouge2': 20.0367, 'rougeL': 19.5339, 'rougeLsum': 19.5339}",0.876700,-0.805100,14.130000
5,1.146300,2.514737,"{'rouge1': 42.185, 'rouge2': 18.7982, 'rougeL': 19.3514, 'rougeLsum': 19.3514}",0.874800,-0.815800,13.900000
6,1.153500,2.519048,"{'rouge1': 42.0433, 'rouge2': 18.2183, 'rougeL': 19.096, 'rougeLsum': 19.096}",0.873600,-0.836500,13.245000
7,1.105000,2.554547,"{'rouge1': 42.6564, 'rouge2': 18.934, 'rougeL': 19.1676, 'rougeLsum': 19.1676}",0.874100,-0.836700,13.820000
8,1.094800,2.590865,"{'rouge1': 44.3364, 'rouge2': 19.3218, 'rougeL': 20.0526, 'rougeLsum': 20.0526}",0.875600,-0.817500,14.030000
9,1.073000,2.599508,"{'rouge1': 43.7072, 'rouge2': 19.3837, 'rougeL': 19.3786, 'rougeLsum': 19.3786}",0.874400,-0.840900,14.035000
10,1.030100,2.573015,"{'rouge1': 42.6338, 'rouge2': 19.083, 'rougeL': 19.0249, 'rougeLsum': 19.0249}",0.873700,-0.846400,13.695000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Trainer is attempting to log a value of "{'rouge1': 42.7473, 'rouge2': 18.8594, 'rougeL': 19.0931, 'rougeLsum': 19.0931}" of type <class 'dict'> for key "eval/rouge" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight',

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Trainer is attempting to log a value of "{'rouge1': 41.182, 'rouge2': 19.5265, 'rougeL': 18.933, 'rougeLsum': 18.933}" of type <class 'dict'> for key "eval/rouge" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

events.out.tfevents.1705337356.3507fbe2081c.761.3:   0%|          | 0.00/21.4k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1705339658.3507fbe2081c.761.4:   0%|          | 0.00/517 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text2text-generation", model="sarahahatee/t5-summarization-one-shot-better-prompt-enriched")

config.json:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [ ]:
# To save a file to use with the task's evaluation python script
from numpy import random
import csv

# have to bring all tensors on the same machine

# prepare a pipeline to use manually for examples

# Create a CSV file to store the outputs
output_file = 'summarization_output_one_shot_better_prompt_enriched.csv'

with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    csv_writer = csv.writer(file)
    csv_writer.writerow(['ID', 'dialogue', 'section_text'])  # Writing headers

    for example in test_dataset:
        summary_text = pipe(f"Extract doctor notes from this patient-doctor dialogue, summarizing all the important medical information like in this example. Dialogue: {subset_for_one_shot[example['section_header']][0]}\nSummary: {subset_for_one_shot[example['section_header']][1]}\nDialogue: {example['dialogue']}\nSummary: ")[0]['generated_text']
        if summary_text.startswith('Summary: '):
          # removing redundant 'Summary: ' at the beginning of generated text
          summary_text = summary_text[9:]
        print(summary_text)

        csv_writer.writerow([example['ID'], example['dialogue'], summary_text])



/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (616 > 512). Running this sequence through the model will result in indexing errors


He has a lot of congestion and coughing.
The two-and-a-half-year-old son and 5-and-a-
This 51y/o white male presented to the clinic today originally for hypertension and med
No known drug allergies.
Macrodantin for urinary tract infection.
Noncontributory.
HPI.
He had a testicle removed in 1975. He does not know what he was
GENERAL: No history of left shoulder injury. GENERAL: No history of osteo
Psoriasis and Psoriasis.
She has a history of Huntington disease and has had a history of memory loss and
He has no alcohol or drug use. He has no history of any tobacco or alcohol use.
He lives in Life Care Center. He is a nonsmoker. He does
She is not taking any over the counter or alternative medicines. She is prescribing Salmeter
He is married. He has two living brothers. One does not speak too much with and does
She has been married for 25 weeks. She has not had a miscarriage for
She is an 83-year-old female who presents to the clinic today originally for hyper
CONDITIONAL: Normal.
She ha